In [2]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import ast
import scipy.cluster.hierarchy as shc
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import string
from IPython.display import Markdown
import textdistance
from fonctions import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.util import skipgrams
from nltk.tokenize import word_tokenize
tqdm.pandas()

In [ ]:
file = '/home/noe/Documents/2022-2023/ENC/Mémoire/Dataframes/PDV_grams.xlsx'
viard = '/home/noe/Documents/2022-2023/ENC/Mémoire/Dataframes/Viard_grams.xlsx'

def delete_items_with_prefix(dictionnaire, prefix):
    keys_to_delete = [key for key in dictionnaire if key.startswith(prefix)]
    for key in keys_to_delete:
        del dictionnaire[key]


def is_single_word(s):
    return isinstance(s, str) and len(s.split()) == 1

def keys_with_single_word_value(dictionary):
    single_word_keys = []
    for key, value in dictionary.items():
        if is_single_word(value):
            single_word_keys.append(key)
    return single_word_keys
def join(liste):
    return ' '.join(liste)

df_pdv = pd.read_excel(file)
df_viard = pd.read_excel(viard)

df = pd.DataFrame({'texts':df_pdv['textes'],'clean_PDV': df_pdv['textes'].apply(clean_text)})
df['compare'] = df['clean_PDV'].apply(join)

df2 = pd.DataFrame({'viard_texts':df_viard['text'],'clean_viard':df_viard['text'].apply(clean_text)})

column_without_brackets = [[string.replace('[', '').replace(']', '').lstrip().rstrip() for string in sublist] for sublist in df2['clean_viard']]

df2['clean'] = column_without_brackets
df2['compare'] = df2['clean'].apply(join)
viard = {}
for i in range(len(df2['compare'].tolist())):
    viard[f'viard_{i+1}'] = df2['compare'].tolist()[i]
dico_un = {}

for i in range(len(df['compare'].tolist())):
    dico_un[f'pdv_{i+1}'] = df['compare'].tolist()[i]

pierre_de_la_vigne = list(dico_un.values())*len(viard)
cles_de_la_vigne = list(dico_un.keys())*len(viard)

jean_viard = copies(list(viard.values()),list(dico_un.values()))
cles_jean = copies(list(viard.keys()),list(dico_un.values()))

tqdm.pandas()
zoe = pd.DataFrame({'viard':jean_viard,'pdv':pierre_de_la_vigne})
viard_lemmes = list(viard.values())
viard_keys = list(viard.keys())
pdv_keys = list(dico_un.keys())
trois = copies(viard_keys,list(dico_un.values()))
quatre = pdv_keys*len(viard_keys)

zoe['indice_viard'] = trois
zoe['indice_pdv'] = quatre
zoe = zoe[['indice_viard','viard','indice_pdv','pdv']]


In [ ]:
x = viard_lemmes

from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)


lemmas = []
for i in tqdm(range(len(x))):
    sentences: List[str] = [x[i]]
    # Get the main object from the model (: data iterator + postprocesor
    from pie_extended.models.lasla.imports import get_iterator_and_processor
    for sentence_group in sentences:
        iterator, processor = get_iterator_and_processor()
        dic = tagger.tag_str(sentence_group, iterator=iterator, processor=processor)
        lemmas.append(dic)

In [ ]:
x = df['compare'].tolist()

from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)


lemmes_pdv = []
for i in tqdm(range(len(x))):
    sentences: List[str] = [x[i]]
    # Get the main object from the model (: data iterator + postprocesor
    from pie_extended.models.lasla.imports import get_iterator_and_processor
    for sentence_group in sentences:
        iterator, processor = get_iterator_and_processor()
        dic = tagger.tag_str(sentence_group, iterator=iterator, processor=processor)
        lemmes_pdv.append(dic)

In [ ]:
#création du dictionnaire de pos

dic1 = [item for sublist in lemmes_pdv for item in sublist]
dic2 = [item for sublist in lemmas for item in sublist]

dic1.extend(dic2)

mots = []
part_of_speech = []
lemmes = []
form = []

for dic in dic1:
    mots.append(dic.get('form'))
    part_of_speech.append(dic.get('pos'))
    lemmes.append(dic.get('lemma'))
    form.append(dic.get('form'))


In [ ]:
dico = pd.DataFrame({'forme':mots,'lemmes':lemmes,'pos':part_of_speech})

In [ ]:
unique = dico['forme'].drop_duplicates().to_frame()

In [ ]:
unique = unique.reset_index()

In [ ]:
indices = unique['index'].tolist()

In [ ]:
dico.to_excel('dico.xlsx')